In [1]:
# Import

import pandas as pd
import os
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score, confusion_matrix
import sklearn as skl
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Load data

df = pd.read_csv(os.path.join('..', 'Resources', 'Created_Files', 'Feature_selected_df.csv'))
df

,Age,Previously_Insured,Vehicle_Damage,Annual_Premium,Vintage,Response
0,23,0,1,29648.0,22,1
1,74,0,1,2630.0,68,0
2,22,0,1,35674.0,116,0
3,26,0,0,27920.0,120,0
4,44,1,0,31630.0,129,0
...,...,...,...,...,...,...
500435,28,0,1,2630.0,135,1
500436,59,0,1,2630.0,237,1
500437,33,1,0,29282.0,273,1
500438,45,0,1,61984.0,184,1


In [3]:
# Define X

X = df.drop(['Response'], axis=1)
X.head(3)

,Age,Previously_Insured,Vehicle_Damage,Annual_Premium,Vintage
0,23,0,1,29648.0,22
1,74,0,1,2630.0,68
2,22,0,1,35674.0,116


In [4]:
# Define y

y = df['Response']
y.value_counts()

1    250220
0    250220
Name: Response, dtype: int64

In [5]:
# Split data into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
# Scale the dataset

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Deep Learning

number_input_feature = len(X_train_scaled[0])
hidden_nodes_layer1 = 8 
hidden_nodes_layer2 = 6
hidden_nodes_layer3 = 4

nn_model = tf.keras.models.Sequential()

nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_feature))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))
nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 48        
                                                                 
 dense_1 (Dense)             (None, 6)                 54        
                                                                 
 dense_2 (Dense)             (None, 4)                 28        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 135
Trainable params: 135
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Checkpoint

# Define the checkpoint path

os.makedirs('Checkpoints/', exist_ok=True)
checkpoint_path = 'Checkpoints/DeepLearning_weights.{epoch:02d}.hdf5'
cp_callback = ModelCheckpoint(filepath = checkpoint_path, verbose = 1, save_weights_only = True, save_freq = 'epoch', period = 50)

In [9]:
# Fit the model

start_time = time.time()

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

end_time = time.time()

Epoch 1/100
11730/11730 [==============================] - 17s 1ms/step - loss: 0.4367 - accuracy: 0.7899
Epoch 2/100
11730/11730 [==============================] - 13s 1ms/step - loss: 0.4323 - accuracy: 0.7912
Epoch 3/100
11730/11730 [==============================] - 13s 1ms/step - loss: 0.4321 - accuracy: 0.7911
Epoch 4/100
11730/11730 [==============================] - 13s 1ms/step - loss: 0.4320 - accuracy: 0.7915
Epoch 5/100
11730/11730 [==============================] - 13s 1ms/step - loss: 0.4319 - accuracy: 0.7915
Epoch 6/100
11730/11730 [==============================] - 14s 1ms/step - loss: 0.4317 - accuracy: 0.7914
Epoch 7/100
11730/11730 [==============================] - 13s 1ms/step - loss: 0.4316 - accuracy: 0.7916
Epoch 8/100
11730/11730 [==============================] - 13s 1ms/step - loss: 0.4316 - accuracy: 0.7914
Epoch 9/100
11730/11730 [==============================] - 13s 1ms/step - loss: 0.4315 - accuracy: 0.7914
Epoch 10/100
11730/11730 [====================

In [10]:
# Evaluation
                     
elapsed_time  = end_time - start_time

print(f'''Deep Learning Performance

Loss: {round(model_loss*100,2)}%

Accuracy: {round(model_accuracy*100,2)}%

Training Time: {round(elapsed_time,1)} Seconds''')

Deep Learning Performance

Loss: 43.45%

Accuracy: 78.93%

Training Time: 1372.6 Seconds
